In [7]:
import numpy as np
import pandas as pd
column_names = ['user_id', 'item_id', 'rating', 'timestamp']
user_rating = pd.read_csv(r"D:\videos pgp\data\ml-100k\u.data",sep="\t",names=column_names)

In [8]:
user_rating.head()

,user_id,item_id,rating,timestamp
0,196,242,3,881250949
1,186,302,3,891717742
2,22,377,1,878887116
3,244,51,2,880606923
4,166,346,1,886397596


In [9]:
movie_titles = pd.read_csv(r"D:\videos pgp\data\ml-100k\Movie_Id_Titles")
movie_titles.head()

,item_id,title
0,1,Toy Story (1995)
1,2,GoldenEye (1995)
2,3,Four Rooms (1995)
3,4,Get Shorty (1995)
4,5,Copycat (1995)


In [10]:
user_rating = pd.merge(user_rating,movie_titles,on='item_id')
user_rating.head()

,user_id,item_id,rating,timestamp,title
0,196,242,3,881250949,Kolya (1996)
1,63,242,3,875747190,Kolya (1996)
2,226,242,5,883888671,Kolya (1996)
3,154,242,3,879138235,Kolya (1996)
4,306,242,5,876503793,Kolya (1996)


In [12]:
n_users = user_rating.user_id.nunique()
n_items = user_rating.item_id.nunique()

print('Num. of Users: '+ str(n_users))
print('Num of Movies: '+str(n_items))

Num. of Users: 943
Num of Movies: 1682


In [14]:
from sklearn.model_selection import train_test_split
train_data, test_data = train_test_split(user_rating, test_size=0.3)

In [17]:
train_data.shape
test_data.shape

(30000, 5)

In [29]:
train_data_matrix = np.zeros((n_users,n_items))
for line in train_data.itertuples():
    train_data_matrix[line[1]-1,line[2]-1]=line[3]
train_data_matrix.shape
test_data_matrix = np.zeros((n_users,n_items))
for line in test_data.itertuples():
    test_data_matrix[line[1]-1,line[2]-1]=line[3]
test_data_matrix.shape

(943, 1682)

In [43]:
from sklearn.metrics import pairwise_distances
user_similarity = pairwise_distances(train_data_matrix,metric="cosine")
item_similarity = pairwise_distances(train_data_matrix.T,metric="cosine")
item_similarity.shape

(1682, 1682)

In [44]:
def predict(rating,similarity,type='user'):
    if type == 'user':
        mean_user_rating = rating.mean(axis=1)
        ratings_diff = (rating-mean_user_rating[:,np.newaxis])
        pred = mean_user_rating[:,np.newaxis]+(similarity.dot(ratings_diff)/np.array([np.abs(similarity).sum(axis=1)]).T)
    elif type == 'item':
         pred = rating.dot(similarity) / np.array([np.abs(similarity).sum(axis=1)]) 
    return pred

In [45]:
item_prediction= predict(train_data_matrix,item_similarity,"item")
user_prediction= predict(train_data_matrix,user_similarity,"user")

In [47]:
from sklearn.metrics import mean_squared_error
from math import sqrt
def rmse(prediction,ground_truth):
    prediction = prediction[ground_truth.nonzero()].flatten()
    ground_truth = ground_truth[ground_truth.nonzero()].flatten()
    return sqrt(mean_squared_error(prediction,ground_truth))

In [48]:
print('User-based CF RMSE: ' + str(rmse(user_prediction, test_data_matrix)))
print('Item-based CF RMSE: ' + str(rmse(item_prediction, test_data_matrix)))

User-based CF RMSE: 3.1598475436367965
Item-based CF RMSE: 3.4669709758576843
